In [3]:
## Converting all files from a stream folder to csv tab

## A tweet flattening function
import json
def flatten_tweets(path):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    with open(path, 'r') as fh:
        tweets_json = fh.read().split("\n")

    # Iterate through each tweet
    for tweet in tweets_json:
        if len(tweet) > 0:
            tweet_obj = json.loads(tweet)

        # Store the user screen name in 'user-screen_name'
            tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']

            # Check if this is a 140+ character tweet
            if 'extended_tweet' in tweet_obj:
                # Store the extended tweet text in 'extended_tweet-full_text'
                tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']

            if 'retweeted_status' in tweet_obj:
                # Store the retweet user screen name in 'retweeted_status-user-screen_name'
                tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

                # Store the retweet text in 'retweeted_status-text'
                tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']

                if 'extended_tweet' in tweet_obj['retweeted_status']:
                    tweet_obj['retweeted_status-extended_tweet-full_text'] = tweet_obj['retweeted_status']['extended_tweet']['full_text']

            if 'quoted_status' in tweet_obj:
                tweet_obj['quoted_status-text'] = tweet_obj['quoted_status']['text']

                if 'extended_tweet' in tweet_obj['quoted_status']:
                    tweet_obj['quoted_status-extended_tweet-full_text']=tweet_obj['quoted_status']['extended_tweet']['full_text']

            tweets_list.append(tweet_obj)
    return tweets_list


## Converting all files from a stream folder to csv tab
# Import pandas and glob
import pandas as pd
import glob

# Give the path of the folder containing all the streaming json files

folder_path = r"C:\Users\gabri\OneDrive\Desktop\temp"
print('The folder path were all the streaming are is :', folder_path)

print('\n','----------------------------------------------','\n')
print('The file conversion started, this operation can take time \n')

# Flatten each streaming file into a python object (list) and upgrade the initial list with all the other dictionnary.
path_list=glob.glob(folder_path+'/*.json')
tweets= flatten_tweets(path_list[0])
#n=len(path_list)//15
#if n>1 :#there is more than one streaming file in the folder

    #for i in range(1,len(path_list)):
        #tweets_i = flatten_tweets(path_list[i])
        #tweets+= tweets_i

# Create a DataFrame from `tweets`
ds_tweets = pd.DataFrame(tweets)

# Drop all the text duplicate
ds_tweets = ds_tweets.drop_duplicates(subset = ['text'])

# Print out the first 5 tweets text from this dataset
print('The first 5 tweets text from this dataset are \n')
print(ds_tweets['text'].values[0:5])
print('\n','----------------------------------------------','\n')
## Passing the index to datetime object

# Print created_at to see the original format of datetime in Twitter data
#print('We print created_at to see the original format of datetime in Twitter data')
#print(ds_tweets['created_at'].head())
#print('\n')
#Convert the created_at column to np.datetime object
ds_tweets['created_at'] = pd.to_datetime(ds_tweets['created_at'])

# Print created_at to see new format
#print('We print created_at to see new format')
#print(ds_tweets['created_at'].head())
#print('\n')
# Set the index of ds_tweets to created_at
ds_tweets = ds_tweets.set_index('created_at')
print('We print the head of data frame')
print(ds_tweets.head(-2)) #show the first 5 and last 5 rows of the dataframe

## Selecting the streaming date
import datetime as dt
import numpy as np
import pytz

# Enter the start date and end date (year, month, day, hour, minute,seconds, microseconds) (you have to specify always the sec and micsec)
#start_date = dt.datetime(2020, 8, 24, 15, 0, 0, tzinfo=dt.timezone.utc)
#end_date = dt.datetime(2020, 8, 24, 22, 0, 0, tzinfo=dt.timezone.utc)
#print('La période de stream séléctionnée est entre le %s et le %s'%(start_date,end_date))

#greater than the start date and smaller than the end date
#mask = (ds_tweets.index > start_date) & (ds_tweets.index <= end_date)
#ds_tweets = ds_tweets.loc[mask] # We replace the complete dataframe with a dataframe containing only the slected dates

print('\n','----------------------------------------------','\n')
print('On passe à la recherche des mots les plus fréquents')
print('\n','----------------------------------------------','\n')
## Finding the most frequent words
ds_tweets_text = ds_tweets[['text',
                            'extended_tweet-full_text',
                            'quoted_status-text',
                            'quoted_status-extended_tweet-full_text',
                            'retweeted_status-text',
                            'retweeted_status-extended_tweet-full_text'
                           ]]

text = ""
nb_row, nb_col = ds_tweets_text.shape
for i in range(nb_row):
    for j in range(nb_col):
        text+=str(ds_tweets_text.iloc[i,j])
import nltk
print('On vérifie le téléchargement de certaines bibiliothèques et on les télécharge sinon.')
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter


tokens = [w for w in word_tokenize(text.lower())if w.isalpha()]
#words to suppress
stop_words = stopwords.words('english') + stopwords.words('spanish')+stopwords.words('french')+['https', 'amp']
no_stops = [t for t in tokens if t not in stop_words]

word_count_processed_text = Counter(no_stops)
word_frequency_table_processed_text = pd.DataFrame(word_count_processed_text.most_common())
word_frequency_table_processed_text.columns = ['Word','Frequency']
print('Voici les 10 mots les plus fréquents')
print(word_frequency_table_processed_text[:10])

print('\n','----------------------------------------------','\n')
print("On passe à l'analyse des sentiments")
print('\n','----------------------------------------------','\n')
## Sentiment Analysis
'''Sentiment analysis provides us a small glimpse of the meaning of texts with a rather directly interpretable method. While it has its limitations, it's a good place to begin working with textual data. There's a number of out-of-the-box tools in Python we can use for sentiment analysis.'''
import nltk
nltk.download('vader_lexicon')

# Load SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate new SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Generate sentiment scores
sentiment_scores = ds_tweets['text'].apply(sid.polarity_scores)
ds_tweets['sentiment_global']= sentiment_scores
print('\n')
column_name = ds_tweets.columns.values
print('Le nom des colonnes est', column_name)
print('\n')
print('\n')

sentiment_global_index =np.where(column_name == 'sentiment_global')
sentiment_global_index = int(sentiment_global_index[0])
ds_tweets['sentiment_compound']=pd.DataFrame([0 for i in range(len(ds_tweets))])
for i in range(len(ds_tweets)):
    if 'compound' in ds_tweets.iloc[i,sentiment_global_index]:
        dico_sentiment = ds_tweets.iloc[i,sentiment_global_index]
        ds_tweets.iloc[i,sentiment_global_index + 1]=dico_sentiment['compound']
print('Voici la table sélectionnée avec le sentiment associé à chaque tweet en dernière colonne')
print(ds_tweets.head())
print('\n','----------------------------------------------','\n')
print("On passe au regroupement des tweets par mots pour avoir un sentiment pour chaque mot de la liste")
print('\n','----------------------------------------------','\n')


0:00      False                    NaN   
2020-12-06 18:56:50+00:00      False                    NaN   
...                              ...                    ...   
2020-12-06 18:56:54+00:00      False                    NaN   
2020-12-06 18:56:54+00:00      False                    NaN   
2020-12-06 18:56:54+00:00      False                    NaN   
2020-12-06 18:56:55+00:00      False                    NaN   
2020-12-06 18:56:55+00:00      False                    NaN   

                          in_reply_to_status_id_str  in_reply_to_user_id  \
created_at                                                                 
2020-12-06 18:56:50+00:00                      None                  NaN   
2020-12-06 18:56:50+00:00                      None                  NaN   
2020-12-06 18:56:50+00:00                      None                  NaN   
2020-12-06 18:56:50+00:00                      None                  NaN   
2020-12-06 18:56:50+00:00                      None         

In [4]:
ds_tweets.head()

,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,...,quoted_status_permalink,quoted_status-text,quoted_status-extended_tweet-full_text,extended_entities,possibly_sensitive,extended_tweet,extended_tweet-full_text,display_text_range,sentiment_global,sentiment_compound
created_at,,,,,,,,,,,,,,,,,,,,,
2020-12-06 18:56:50+00:00,1335659481880031240,1335659481880031240,RT @Weinsteinlaw: Joe Biden received a greater...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'comp...",0.3612
2020-12-06 18:56:50+00:00,1335659482001494016,1335659482001494016,"RT @DFBHarvard: Frankly, my dear audience, Chr...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.176, 'neu': 0.729, 'pos': 0.095, 'co...",-0.4968
2020-12-06 18:56:50+00:00,1335659481888419842,1335659481888419842,Comedy News Network - The Most Laughable Headl...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,...,"{'url': 'https://t.co/p8sCCnEh7M', 'expanded':...",Joe Biden's already more popular than Presiden...,Joe Biden's already more popular than Presiden...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.096, 'neu': 0.589, 'pos': 0.315, 'co...",0.4561
2020-12-06 18:56:50+00:00,1335659482106359809,1335659482106359809,RT @TeamTrump: President @realDonaldTrump: We ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,...,NaN,NaN,NaN,"{'media': [{'id': 1335381629926301697, 'id_str...",False,NaN,NaN,NaN,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...",0.8452
2020-12-06 18:56:50+00:00,1335659482131673090,1335659482131673090,Those men and women should be arrested!,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,...,"{'url': 'https://t.co/m20YtMqxxW', 'expanded':...","UnAmerican, post-democracy rubes for a failed ...",NaN,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.361, 'neu': 0.639, 'pos': 0.0, 'comp...",-0.5255


In [5]:
ds_tweets.columns

Index(['id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'retweeted_status',
       'is_quote_status', 'quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
       'lang', 'timestamp_ms', 'user-screen_name',
       'retweeted_status-user-screen_name', 'retweeted_status-text',
       'retweeted_status-extended_tweet-full_text', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
       'quoted_status-text', 'quoted_status-extended_tweet-full_text',
       'extended_entities', 'possibly_sensitive', 'extended_tweet',
       'extended_tweet-full_text', 'display_text_range', 'sentiment_global',
       'sentiment_compound'],
      dtype='object')

In [6]:
ds_tweets['place']


created_at
2020-12-06 18:56:50+00:00    None
2020-12-06 18:56:50+00:00    None
2020-12-06 18:56:50+00:00    None
2020-12-06 18:56:50+00:00    None
2020-12-06 18:56:50+00:00    None
                             ... 
2020-12-06 18:56:54+00:00    None
2020-12-06 18:56:55+00:00    None
2020-12-06 18:56:55+00:00    None
2020-12-06 18:56:55+00:00    None
2020-12-06 18:56:55+00:00    None
Name: place, Length: 90, dtype: object

In [7]:
data_place_not_null = ds_tweets[~ds_tweets['user'].isna()]['user']
len(data_place_not_null)

90

In [99]:
data_place_not_null.iloc[5]

{'id': 790821959676284928,
 'id_str': '790821959676284928',
 'name': 'Danny',
 'screen_name': 'YaBoiD4nny',
 'location': 'Sydney, New South Wales',
 'url': None,
 'description': 'the retweet king',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 129,
 'friends_count': 422,
 'listed_count': 2,
 'favourites_count': 37631,
 'statuses_count': 30860,
 'created_at': 'Tue Oct 25 07:47:02 +0000 2016',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'profile_image_url': 'http://pbs.twimg.com/profile_images/1265880944554078210/NmO3Q

In [ ]:
## Création de la data Frame pour le clustering.

In [8]:

import numpy as np

In [9]:
def calculateCentroid(place):
    """ Calculates the centroid from a bounding box."""
    # Obtain the coordinates from the bounding box.
    coordinates = place['bounding_box']['coordinates'][0]
    
    longs = np.unique( [x[0] for x in coordinates] )
    lats  = np.unique( [x[1] for x in coordinates] )
    
    if len(longs) == 1 and len(lats) == 1:
        # return a single coordinate
        return (longs[0], lats[0])
    elif len(longs) == 2 and len(lats) == 2:
        # If we have two longs and lats, we have a box.
        central_long = np.sum(longs) / 2
        central_lat  = np.sum(lats) / 2
    else:
        raise ValueError("Non-rectangular polygon not supported.")

    return (central_long, central_lat)



In [11]:
# Load SentimentIntensityAnalyzer

def clustering_data(dataframe):
    """Select the variable for the clustering from the dataframe:
    'dataframe', it makes a copy of the dataframe """
    dfcopy=pd.DataFrame(dataframe['id'])
    
    dfcopy['truncated']=dataframe['truncated']

    dfcopy['text-sentiment_global']=dataframe['sentiment_global']
    dfcopy['text-sentiment_compound']=dataframe['sentiment_compound']


    # User 
    dfcopy['user-location']= None
    dfcopy['user-description']= None
    dfcopy['user-protected']= None
    dfcopy['user-verified']= None
    dfcopy['user-followers_count']= None 
    dfcopy['user-friends_count']= None
    dfcopy['user-listed_count']= None
    dfcopy['user-favourites_count']= None
    dfcopy['user-statuses_count']= None
    dfcopy['user-created_at']= pd.to_datetime([None]*len(dfcopy), utc=True)
    dfcopy['user-utc_offset']= None 
    dfcopy['user-time_zone']= None 
    dfcopy['user-contributors_enabled']= None 
    dfcopy['user-geo_enabled']= None 
    dfcopy['user-profile_background_color']= None 
    dfcopy['user-profile_sidebar_border_color']= None 
    dfcopy['user-profile_sidebar_fill_color']= None 
    dfcopy['user-profile_text_color']= None 
    dfcopy['user-profile_use_background_image']= None 
    dfcopy['user-default_profile']= None 
    dfcopy['user-default_profile_image']= None 
    dfcopy['user-following']= None 
    dfcopy['user-follow_request_sent']= None 
    dfcopy['user-notifications']= None 

    # Place
    dfcopy['place-place_type']=None
    dfcopy['place-name']=None
    dfcopy['place-full_name']=None
    dfcopy['place-country_code']= None
    dfcopy['place-country']=None
    dfcopy['place-bounding_box']=None
    dfcopy['place-attributes']=None


    # Instantiate new SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()

    for i in range(len(dfcopy)):         
        #Dans le dictionnaire user
        if dataframe['user'][i] != None :

            dfcopy['user-location'][i]=dataframe['user'][i]['location']
            dfcopy['user-description'][i]=dataframe['user'][i]['description']
            dfcopy['user-protected'][i]=dataframe['user'][i]['protected']
            dfcopy['user-verified'][i]=dataframe['user'][i]['verified']
            dfcopy['user-followers_count'][i]=dataframe['user'][i]['followers_count']
            dfcopy['user-friends_count'][i]=dataframe['user'][i]['friends_count']
            dfcopy['user-listed_count'][i]=dataframe['user'][i]['listed_count']
            dfcopy['user-favourites_count'][i]=dataframe['user'][i]['favourites_count']
            dfcopy['user-statuses_count'][i]=dataframe['user'][i]['statuses_count']
            dfcopy['user-created_at'][i]=pd.to_datetime(dataframe['user'][i]['created_at'])
            dfcopy['user-utc_offset'][i]=dataframe['user'][i]['utc_offset']
            dfcopy['user-time_zone'][i]=dataframe['user'][i]['time_zone']
            dfcopy['user-contributors_enabled'][i]=dataframe['user'][i]['contributors_enabled']
            dfcopy['user-geo_enabled'][i]=dataframe['user'][i]['utc_offset']
            dfcopy['user-profile_background_color'][i]=dataframe['user'][i]['profile_background_color']
            dfcopy['user-profile_sidebar_border_color'][i]=dataframe['user'][i]['profile_sidebar_border_color']
            dfcopy['user-profile_sidebar_fill_color'][i]=dataframe['user'][i]['profile_sidebar_fill_color']
            dfcopy['user-profile_text_color'][i]=dataframe['user'][i]['profile_text_color']
            dfcopy['user-profile_use_background_image'][i]=dataframe['user'][i]['profile_use_background_image']
            dfcopy['user-default_profile'][i]=dataframe['user'][i]['default_profile']
            dfcopy['user-default_profile_image'][i]=dataframe['user'][i]['profile_link_color']
            dfcopy['user-following'][i]=dataframe['user'][i]['following']
            dfcopy['user-follow_request_sent'][i]=dataframe['user'][i]['follow_request_sent']
            dfcopy['user-notifications'][i]=dataframe['user'][i]['notifications']

           


        
        if dataframe['place'][i] != None:
            dfcopy['place-place_type'][i]=dataframe['place'][i]['place_type']
            dfcopy['place-name'][i]=dataframe['place'][i]['name']
            dfcopy['place-full_name'][i]=dataframe['place'][i]['full_name']
            dfcopy['place-country_code'][i]=dataframe['place'][i]['country_code']
            dfcopy['place-country'][i]=dataframe['place'][i]['country']
            dfcopy['place-bounding_box'][i]=dataframe['place'][i]['bounding_box']
            dfcopy['place-attributes'][i]=dataframe['place'][i]['attributes']
            
            
    # Generate sentiment scores for the description 
        #dfcopy.dropna(subset=['user-description'],inplace=True)
        #sentiment_scores = dfcopy['user-description'].apply(sid.polarity_scores)
        #dfcopy['user-description-sentiment_global']= sentiment_scores

    return dfcopy
    
    
dfcaca= clustering_data(ds_tweets)






    
    





In [12]:
# Generate sentiment scores for the description 
dfcaca.dropna(subset=['user-description'],inplace=True)
sentiment_scores = dfcaca['user-description'].apply(sid.polarity_scores)
dfcaca['user-description-sentiment_global']= sentiment_scores

In [124]:
dfcaca[dfcaca['place-place_type'].isna() == False]

,id,truncated,text-sentiment_global,text-sentiment_compound,user-location,user-description,user-protected,user-verified,user-followers_count,user-friends_count,...,user-follow_request_sent,user-notifications,place-place_type,place-name,place-full_name,place-country_code,place-country,place-bounding_box,place-attributes,user-description-sentiment_global
created_at,,,,,,,,,,,,,,,,,,,,,
2020-11-04 11:15:52+00:00,1323947065119068161,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,None,LFGM,False,False,133,1334,...,None,None,city,Queens,"Queens, NY",US,United States,"{'type': 'Polygon', 'coordinates': [[[-73.9625...",{},"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2020-11-04 11:15:53+00:00,1323947069208436736,True,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.7184,European Union,I have seen the future & this is how it begins...,False,False,456,1235,...,None,None,city,Carlton,"Carlton, England",GB,United Kingdom,"{'type': 'Polygon', 'coordinates': [[[-1.16777...",{},"{'neg': 0.443, 'neu': 0.557, 'pos': 0.0, 'comp..."
2020-11-04 11:15:55+00:00,1323947077836177409,False,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp...",0.5994,"Connecticut, USA",hey bro,False,False,4,47,...,None,None,city,Norwich,"Norwich, CT",US,United States,"{'type': 'Polygon', 'coordinates': [[[-72.1751...",{},"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2020-11-04 11:15:56+00:00,1323947081690734592,True,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Arlon - Belgique,Journaliste La Meuse Luxembourg (Sudpresse),False,False,1417,1097,...,None,None,city,Arlon,"Arlon, Belgique",BE,Belgique,"{'type': 'Polygon', 'coordinates': [[[5.678485...",{},"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2020-11-04 11:15:58+00:00,1323947089982820352,False,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",0.5719,between psychotic & iconic,"new beginnings 🥂 | jackson,ms ✈️ htx | female ...",False,False,769,674,...,None,None,city,Houston,"Houston, TX",US,United States,"{'type': 'Polygon', 'coordinates': [[[-95.8232...",{},"{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-04 11:20:02+00:00,1323948113707618306,False,"{'neg': 0.0, 'neu': 0.427, 'pos': 0.573, 'comp...",0.3989,"Kingston, Jamaica","Passionate #changemaker, Committed to vibrant ...",False,False,11795,836,...,None,None,country,Jamaica,Jamaica,JM,Jamaica,"{'type': 'Polygon', 'coordinates': [[[-78.3688...",{},"{'neg': 0.0, 'neu': 0.48, 'pos': 0.52, 'compou..."
2020-11-04 11:20:05+00:00,1323948126776995840,True,"{'neg': 0.11, 'neu': 0.7, 'pos': 0.19, 'compou...",0.3818,Virginia,Prof @RegentLaw / Torts / Law&Rel /\nParler: @...,False,False,696,662,...,None,None,city,Virginia Beach,"Virginia Beach, VA",US,United States,"{'type': 'Polygon', 'coordinates': [[[-76.2280...",{},"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2020-11-04 11:20:05+00:00,1323948126974169089,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"Pelotas e Cotia, Brasil","Eu nem queria nascer, as freiras disseram que ...",False,False,238,365,...,None,None,city,Cotia,"Cotia, Brasil",BR,Brasil,"{'type': 'Polygon', 'coordinates': [[[-47.0847...",{},"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [130]:
dfcaca.to_csv(path_or_buf= path_list[0] + '.csv', sep = ';')